In [1]:
from preprocess import *

#from fastdtw import fastdtw
from dtaidistance import dtw

import umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

C:\ProgramData\Anaconda3\envs\temporal_event_corr\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\ProgramData\Anaconda3\envs\temporal_event_corr\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
df = pd.read_csv('../data/2019-08-01_test2.csv')
A = get_aggregated(df, '2019-08-01 12:00:00','2019-08-01 12:30:00')
B = A[A['timestamp', 'count'] > 5]

signals = [get_series((B.iloc[x, 0]-min(B.iloc[x, 0])).astype(np.int32), (1+max(B.iloc[x, 0])-min(B.iloc[x, 0])).astype(np.int32))
           for x in range(0, len(B))]

names = [B.iloc[x,:].name
           for x in range(0, len(B))]

Xtr, Xts, Ytr, Yts = train_test_split(signals, list(range(len(signals))), test_size=0.1)

pairwise = np.zeros([len(signals),len(signals)])
#faster = [np.array(x, dtype=np.double) for x in range(0, len(Xtr))]
for x in range(0, 10):
    for y in range(x+1, 10):
        distance = dtw.distance((signals[x]), (signals[y]), penalty=1, window=60) # optimize
        pairwise[x, y] = distance
        pairwise[y, x] = distance

In [ ]:
clusterable_embedding = umap.UMAP(
    n_neighbors=30,
    min_dist=0.0,
    n_components=2,
    random_state=42,
).fit_transform(distance)


plt.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1],
            c=list(range(0,100)), s=0.1, cmap='Spectral')

labels = hdbscan.HDBSCAN(
    min_samples=3,
    min_cluster_size=5,
).fit_predict(clusterable_embedding)

clustered = (labels >= 0)
plt.scatter(clusterable_embedding[~clustered, 0],
            clusterable_embedding[~clustered, 1],
            c=(0.5, 0.5, 0.5),
            #s=0.1,
            alpha=0.5)
plt.scatter(clusterable_embedding[clustered, 0],
            clusterable_embedding[clustered, 1],
            c=labels[clustered],
            #s=0.1,
            cmap='Spectral')
